In [1]:
import sys

sys.path.append('../')
import rings2cosmo_Mamon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from astropy import constants as const
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
Hubble = 67.3
cosmo = FlatLambdaCDM(H0 = Hubble, Om0 = 0.315)

In [2]:
# Physical constants:
c = (const.c).to(u.km/u.second)
clight = c.value

## Important Functions

In [3]:
def Lambda_function(x):
    """Eq. (15) from arXiv:0907.4992v2

    Args:
        x (float): parameter

    Returns:
        float: ratio of gamma funcions.
    """
    return sp.special.gamma((x - 1) / 2) / sp.special.gamma(x / 2)

$$
\mu_E \equiv  \frac{1}{1+\gamma}\frac{1}{\sqrt{\pi}\lambda (\alpha)} c^2\frac{D_S}{D_{LS}}\frac{D_L^{\alpha - 2}}{\theta_E^{1-\alpha}}$$

In [4]:
def virial_mass (DS, DL, DLS, theta_E, gamma, alpha):
    gamma_term = 1.0/(1.0 + gamma)
    dinamics = Lambda_function(alpha)*(np.sqrt(np.pi))
    distance_ratio = DS/DLS
    Ering_ratio = (DL**(alpha - 2))/(theta_E**(1 - alpha))
    return (clight**2)*gamma_term*distance_ratio*Ering_ratio/dinamics

$$r_V = \left( \frac{2\mu_E}{\Delta H_0^2} \right)^{1/\alpha}$$

In [5]:
def virial_radii (DS, DL, DLS, theta_E, gamma, alpha):
    MuE = virial_mass (DS, DL, DLS, theta_E, gamma, alpha)
    D = 200
    return (2*MuE/(D*Hubble))**(1/alpha)
    

$$b = D_L\tilde{\sigma}_{atm}$$

In [6]:
def b_func (DL, seeing_atm, theta_ap):
    chi = theta_ap/seeing_atm
    tilde_sigma = seeing_atm *np.sqrt(1.0 + (chi ** 2.0) / 4.0 + (chi ** 4.0) / 40.0)  # Eq. (20)
    
    return tilde_sigma*DL

## Adding the ctes that I havent added so far

In [8]:
def Weight_normalization(seeing_atm, theta_ap,DL, theta_E, delta):
    ## \tilde{\sigma}
    chi = theta_ap/seeing_atm

    tilde_sigma = seeing_atm * \
        np.sqrt(1 + (chi ** 2) / 4 + (chi ** 4) / 40)  # Eq. (20)
    
    RE = DL*theta_E

    #term B\left ( \frac{\delta - 1}{2}, \frac{1}{2} \right )
    #x = (delta - 1)/2
    euler_beta = Lambda_function(delta)*np.sqrt(np.pi)

    #term  \Gamma\left ( \frac{3 - \delta}{2} \right )
    y = (3 - delta)/2
    euler_gamma = sp.special.gamma(y)

    #term \left(\frac{2\tilde{\sigma}^2_{atm}}{\theta_E^2}\right)
    tilde_sigma_by_theta_E = 2*((tilde_sigma/theta_E)**2)

    return euler_beta*(RE**(2*y))*(tilde_sigma_by_theta_E**y)*euler_gamma/2

$$
\left[\frac{GM_E}{R_E}\right] = \frac{c^2}{2(1+\gamma)}\frac{D_S}{D_{LS}}\theta_E
$$

In [9]:
def Relativistic_term (DS, DLS, theta_E, gamma):
    return (clight**2.0)*(DS/DLS)*theta_E/(2.0*(1.0+gamma))

$$
C_M = \left[\frac{GM_E}{R_E}\right]\frac{2}{\sqrt{\pi}\lambda(\alpha)\xi(\xi - 1)}\left(\frac{1}{R_E}\right)^{2 - \alpha}
$$

In [10]:
def mamon_parameters (DL, DS, DLS, theta_E, alpha, delta,gamma):
    xi = delta + alpha - 2
    R_E = theta_E*DL
    
    rel_term = Relativistic_term (DS, DLS, theta_E, gamma)
    dim = 2.0/(np.sqrt(np.pi)*Lambda_function(alpha)*xi*(xi - 1)*(R_E**(2 - alpha)))
    
    return rel_term*dim

In [11]:
import VDmod

def vel(z_S, z_L, theta_E, seeing_atm, theta_ap, virial_frac, alpha, delta, gamma):
    DS = cosmo.angular_diameter_distance(z_S).value
    DL = cosmo.angular_diameter_distance(z_L).value
    DLS = cosmo.angular_diameter_distance_z1z2(z_L, z_S).value
    xi = delta + alpha - 2
    
    b = b_func (DL, seeing_atm, theta_ap)
    ra = virial_frac*virial_radii (DS, DL, DLS, theta_E, gamma, alpha)
    integral = VDmod.integralVD(b,alpha,delta,ra)

    
    C_mamon_term = mamon_parameters (DL, DS, DLS, theta_E, alpha, delta,gamma)
    weigth_term = Weight_normalization(seeing_atm, theta_ap,DL, theta_E, delta)
    vel = C_mamon_term*integral/weigth_term
    
    return np.sqrt(np.abs(vel))
 


In [12]:
vel_vec = np.vectorize(vel)

In [14]:
data = pd.read_csv('../data/sample80.csv')

z_L = data['zl'].values
z_S = data['zs'].values
theta_E = (data['theta_E'].values * rings2cosmo_Mamon.u.arcsec).to(rings2cosmo_Mamon.u.rad).value
theta_ap = (data['theta_ap'].values * rings2cosmo_Mamon.u.arcsec).to(rings2cosmo_Mamon.u.rad).value
seeing_atm = (data['sigma_atm'].values * rings2cosmo_Mamon.u.arcsec).to(rings2cosmo_Mamon.u.rad).value
velDisp = data['sigma_ap'].values
velDispErr = data['d_sigma_ap'].values

#About the parameters value and distribution
alpha = 2.0
beta = 0.18
delta = 2.40

gamma = 1.0
virial_frac = 0.18
vel_vec(z_S, z_L, theta_E, seeing_atm, theta_ap, virial_frac, alpha, delta, gamma)

array([236.7610242 , 218.81781978, 260.83024563, 226.57127387,
       246.77044455, 229.79737049, 202.82600156, 232.15713731,
       221.79131035, 264.74057289, 257.38697345, 232.08295603,
       231.70975355, 201.38007241, 239.64980257, 205.79078226,
       222.25270664, 244.26947296, 261.02674755, 268.95606875,
       234.63670033, 250.653489  , 260.86024697, 220.56392201,
       259.44365878, 232.79692914, 191.27187191, 198.95469597,
       226.24867672, 274.70284373, 222.99096278, 262.0310447 ,
       247.66130764, 260.46683494, 284.62915645, 206.19247828,
       277.79971678, 234.43677777, 222.83709832, 262.78159409,
       197.16111514, 229.68884296, 184.06099419, 230.63530102,
       227.10731371, 200.25695034, 290.9798065 , 249.42674103,
       195.09464693, 225.80400783, 229.32630317, 196.8061787 ,
       282.20042441, 293.9401261 , 230.61129483, 268.5253392 ,
       266.33223104, 199.44122656, 259.05797847, 238.49752974,
       230.84859774, 260.06264162, 252.65104299, 246.41

In [22]:
import timeit

In [15]:
%%time

virial_frac = 0.18
vel_vec(z_S, z_L, theta_E, seeing_atm, theta_ap, virial_frac, alpha, delta, gamma)

CPU times: user 443 ms, sys: 6.7 s, total: 7.14 s
Wall time: 17.7 s


array([236.7610242 , 218.81781978, 260.83024563, 226.57127387,
       246.77044455, 229.79737049, 202.82600156, 232.15713731,
       221.79131035, 264.74057289, 257.38697345, 232.08295603,
       231.70975355, 201.38007241, 239.64980257, 205.79078226,
       222.25270664, 244.26947296, 261.02674755, 268.95606875,
       234.63670033, 250.653489  , 260.86024697, 220.56392201,
       259.44365878, 232.79692914, 191.27187191, 198.95469597,
       226.24867672, 274.70284373, 222.99096278, 262.0310447 ,
       247.66130764, 260.46683494, 284.62915645, 206.19247828,
       277.79971678, 234.43677777, 222.83709832, 262.78159409,
       197.16111514, 229.68884296, 184.06099419, 230.63530102,
       227.10731371, 200.25695034, 290.9798065 , 249.42674103,
       195.09464693, 225.80400783, 229.32630317, 196.8061787 ,
       282.20042441, 293.9401261 , 230.61129483, 268.5253392 ,
       266.33223104, 199.44122656, 259.05797847, 238.49752974,
       230.84859774, 260.06264162, 252.65104299, 246.41

In [16]:
%%time

virial_frac = 0.018
vel_vec(z_S, z_L, theta_E, seeing_atm, theta_ap, virial_frac, alpha, delta, gamma)

CPU times: user 454 ms, sys: 6.7 s, total: 7.16 s
Wall time: 18.9 s


array([239.19751148, 221.53030959, 263.83190135, 229.704426  ,
       250.02745293, 233.072887  , 206.06427572, 235.65897823,
       225.57158326, 268.68206592, 261.38300234, 236.07530258,
       235.76419592, 205.49629107, 244.0080318 , 210.05381345,
       226.60120432, 248.71901151, 265.60960345, 273.69664601,
       239.26000316, 255.36897165, 265.71162602, 225.29576961,
       264.38432594, 237.62446139, 195.90956375, 203.67239255,
       231.1214593 , 279.78267708, 227.89707749, 265.2163903 ,
       252.79563167, 265.7073648 , 290.00664518, 209.64622705,
       283.55246112, 240.0018439 , 228.42450386, 268.64503689,
       202.27275459, 234.7504684 , 187.09179394, 236.79264971,
       233.95965542, 204.04707413, 297.56029201, 256.55879077,
       198.28952106, 230.106691  , 233.42090397, 203.6858682 ,
       287.12033441, 301.79264324, 234.14120426, 275.54669598,
       271.39591013, 206.66971465, 267.02462948, 243.35516512,
       235.82981182, 264.47253061, 257.05495887, 253.78